## Deliverable 2. Create a Customer Travel Destinations Map.

In [28]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [29]:
import traceback

In [30]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,55.99,86,84,1.01,broken clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,55.40,44,0,28.77,clear sky
2,2,Port Hardy,CA,50.6996,-127.4199,46.40,87,75,6.91,broken clouds
3,3,Vaini,TO,-21.2000,-175.2000,78.80,94,75,6.91,broken clouds
4,4,Albany,US,42.6001,-73.9662,35.01,62,99,4.74,overcast clouds


In [33]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 90


In [34]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,55.99,86,84,1.01,broken clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,55.40,44,0,28.77,clear sky
2,2,Port Hardy,CA,50.6996,-127.4199,46.40,87,75,6.91,broken clouds
3,3,Vaini,TO,-21.2000,-175.2000,78.80,94,75,6.91,broken clouds
4,4,Albany,US,42.6001,-73.9662,35.01,62,99,4.74,overcast clouds
5,5,Atuona,PF,-9.8000,-139.0333,77.65,79,100,16.13,overcast clouds
7,7,Torbay,CA,47.6666,-52.7314,30.99,80,90,6.91,overcast clouds
8,8,Hermanus,ZA,-34.4187,19.2345,71.01,82,86,8.01,overcast clouds
9,9,Lebu,CL,-37.6167,-73.6500,58.51,58,3,21.54,clear sky
10,10,Port Elizabeth,ZA,-33.9180,25.5701,69.80,88,90,10.36,overcast clouds


In [35]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [36]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [37]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,55.99,broken clouds,-46.1927,168.8643,
1,Punta Arenas,CL,55.40,clear sky,-53.1500,-70.9167,
2,Port Hardy,CA,46.40,broken clouds,50.6996,-127.4199,
3,Vaini,TO,78.80,broken clouds,-21.2000,-175.2000,
4,Albany,US,35.01,overcast clouds,42.6001,-73.9662,
5,Atuona,PF,77.65,overcast clouds,-9.8000,-139.0333,
7,Torbay,CA,30.99,overcast clouds,47.6666,-52.7314,
8,Hermanus,ZA,71.01,overcast clouds,-34.4187,19.2345,
9,Lebu,CL,58.51,clear sky,-37.6167,-73.6500,
10,Port Elizabeth,ZA,69.80,overcast clouds,-33.9180,25.5701,


In [38]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
        
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        #traceback.print_exc()
        pass

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [57]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']

In [58]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mataura,NZ,55.99,broken clouds,-46.1927,168.8643,Ellie's Villa
1,Punta Arenas,CL,55.40,clear sky,-53.1500,-70.9167,Hotel Dreams Del Estrecho
2,Port Hardy,CA,46.40,broken clouds,50.6996,-127.4199,Airport Inn
3,Vaini,TO,78.80,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Atuona,PF,77.65,overcast clouds,-9.8000,-139.0333,Villa Enata
7,Torbay,CA,30.99,overcast clouds,47.6666,-52.7314,Gallows Cove
8,Hermanus,ZA,71.01,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
9,Lebu,CL,58.51,clear sky,-37.6167,-73.6500,Hostal Las Lilas
10,Port Elizabeth,ZA,69.80,overcast clouds,-33.9180,25.5701,39 On Nile Guest House
11,Bluff,NZ,55.99,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park


In [59]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [71]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>

"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [72]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))